In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import datetime

In [8]:
def get_info(link):
    page = requests.get(link)
    soup = bs(page.text, "lxml")
    
    # Initialize all fields as "non spécifié"
    company = title = lieu = contrat = location = salaire = travail = experience = education = "Non Spécifié"
    competences = []
    date = "Non trouvé"
    debut = "non spécifié"
    
    # Try to extract company
    try:
        company = soup.find('a', attrs={'class': "sc-emIrwa Trejm"}).string
    except AttributeError:
        pass  # Keep default value
    
    # Try to extract title
    try:
        title = soup.find('section', attrs={'class': "kjbyhm-1 jLtjFq"}).find_next('h2').text
    except AttributeError:
        pass  # Keep default value

    # Try to extract location (lieu)
    try:
        lieu = soup.find('a', attrs={'class': "sc-hgRRfv jOCZOZ sc-emIrwa euWxDX sc-1cigsri-2 faxiCn"}).text
    except AttributeError:
        pass  # Keep default value
    
    # Try to extract job contract details (contrat)
    try:
        contrat = soup.find_all('div', class_="sc-bOhtcR lhgrLO")[0].text
    except (AttributeError, IndexError):
        pass  # Keep default value

    # Try to extract location, salary, work type, experience, education
    try:
        location = soup.find('i', attrs={'name': "location"}).find_next('span').text
    except AttributeError:
        pass  # Keep default value
    
    try:
        salaire=soup.find('span', attrs = {'class' : "sc-bXCLTC kJcLKT"}).next_sibling.text
        salaire=salaire.replace('\xa0',' ')
    except AttributeError:
        pass  # Keep default value

    try:
        debut = soup.find('i', attrs={'name': "clock"}).find_next('span').find_next('span').text
    except AttributeError:
        pass  # Keep default value
    
    try:
        travail = soup.find('i', attrs={'name': "remote"}).find_next('span').text
    except AttributeError:
        pass  # Keep default value

    try:
        experience = soup.find('i', attrs={'name': "experience"}).find_next('span').text
    except AttributeError:
        pass  # Keep default value

    try:
        education = soup.find('i', attrs={'name': "education"}).find_next('span').text
    except AttributeError:
        pass  # Keep default value
    
    # Extract competences
    spans = soup.find_all('div', class_="sc-bOhtcR caYWTG")
    for span in spans:
        competences.append(span.text)

    # Extract job date
    try:
        date_string = soup.find('div', attrs={'class': "sc-bXCLTC yKqsk"}).find('time').get('datetime')[:10]

# Parse the string into a datetime object
        date_obj = datetime.strptime(date_string, "%Y-%m-%d")

# Format it to the desired date format (e.g., 'Day-Month-Year')
        date = date_obj.strftime("%d-%B-%Y")
    except AttributeError:
        pass  # Keep default value
    
    # Create a list with all job details
    line = [company, title, lieu, location, date, contrat, debut, salaire, travail, competences, experience, education, link]
    return line


In [9]:
def get_links(job):
    links = []
    options = Options()
    options.headless = True  # Run in headless mode (no UI)
    
    driver = webdriver.Chrome(options=options)
    
    try:
        driver.get("https://www.welcometothejungle.com/fr/jobs")
        time.sleep(10)  # Wait for the page to load
        
        input_element = driver.find_element(By.ID, "search-query-field")
        input_element.send_keys(job)
        input_element.send_keys(Keys.RETURN)
        time.sleep(5)  # Allow results to load

        for i in range(2):  # Loop for pagination
            items = driver.find_elements(By.CLASS_NAME, 'ais-Hits-list-item')
            
            # Extract job links from each item
            for item in items:
                link_element = item.find_element(By.CLASS_NAME, 'iPeVkS')
                links.append(link_element.get_attribute("href"))
            
            time.sleep(5)  # Slight delay before going to the next page
            
            # Click next page
            next_buttons = driver.find_elements(By.CSS_SELECTOR, 'a.sc-IqJVf.evElzU')
            if next_buttons:
                next_buttons[-1].click()
                time.sleep(10)  # Wait for the next page to load
            else:
                break  # If no next button, exit loop

    finally:
        driver.quit()  # Ensure the driver is closed properly

    return links


In [10]:
def save_jobs_to_dataframe():
    # Get all job links
    job_links = get_links("data science")
    
    # List to hold all job data
    job_data = []
    
    # Loop over each link and extract job info
    for link in job_links:
        info = get_info(link)
        job_data.append(info)
    
    # Create a DataFrame
    df = pd.DataFrame(job_data, columns=['Company', 'Title', 'Lieu', 'Location', 'Date', 'Contrat', 'Debut','Salaire', 'Travail', 'Competences', 'Experience', 'Education', 'Link'])
    
    # Save to CSV (optional)
    df.to_csv('welcome.csv', index=False)
    
    return df

# Example usage
df = save_jobs_to_dataframe()


In [11]:
df

,Company,Title,Lieu,Location,Date,Contrat,Debut,Salaire,Travail,Competences,Experience,Education,Link
0,Non Spécifié,Product Manager (PM) data science,Non Spécifié,Paris,Non trouvé,Non Spécifié,30 septembre 2024,Non spécifié,Télétravail fréquent,[],Non Spécifié,Non Spécifié,https://www.welcometothejungle.com/fr/companie...
1,Non Spécifié,Manager in Strategy & Data Science -Hong Kong,Non Spécifié,Paris,Non trouvé,Non Spécifié,non spécifié,Non spécifié,Télétravail fréquent,[],Non Spécifié,Non Spécifié,https://www.welcometothejungle.com/fr/companie...
2,Non Spécifié,STAGE 2025 - Stage en data science LLM (ALL GE...,Non Spécifié,Toulouse,Non trouvé,Non Spécifié,non spécifié,Non spécifié,Télétravail non autorisé,[],Non Spécifié,Non Spécifié,https://www.welcometothejungle.com/fr/companie...
3,Non Spécifié,Expert en Intelligence Artificielle et Data Sc...,Non Spécifié,Cesson-Sévigné,Non trouvé,Non Spécifié,non spécifié,Non spécifié,Télétravail fréquent,[],Non Spécifié,Non Spécifié,https://www.welcometothejungle.com/fr/companie...
4,Non Spécifié,Data Science Manager,Non Spécifié,Paris,Non trouvé,Non Spécifié,non spécifié,Non spécifié,Télétravail fréquent,[],Non Spécifié,Non Spécifié,https://www.welcometothejungle.com/fr/companie...
5,Non Spécifié,Data Science manager,Non Spécifié,Paris,Non trouvé,Non Spécifié,non spécifié,Non spécifié,Télétravail fréquent,[],Non Spécifié,Non Spécifié,https://www.welcometothejungle.com/fr/companie...
6,Non Spécifié,Responsable Data Science et Insights H/F,Non Spécifié,Paris,Non trouvé,Non Spécifié,non spécifié,70K à 90K €,Télétravail non autorisé,[],Non Spécifié,Non Spécifié,https://www.welcometothejungle.com/fr/companie...
7,Non Spécifié,Stages Data Science et IA H/F,Non Spécifié,Saint-Aubin,Non trouvé,Non Spécifié,non spécifié,Non spécifié,Télétravail non autorisé,[],Non Spécifié,Non Spécifié,https://www.welcometothejungle.com/fr/companie...
8,Non Spécifié,Global RGM Data Science Lead (M/F),Non Spécifié,Neuilly-sur-Seine,Non trouvé,Non Spécifié,non spécifié,Non spécifié,Télétravail non autorisé,[],Non Spécifié,Non Spécifié,https://www.welcometothejungle.com/fr/companie...
9,Non Spécifié,DATA SCIENCE & CONTRÔLE DES PRATIQUES COMMERCI...,Non Spécifié,Paris,Non trouvé,Non Spécifié,non spécifié,Non spécifié,Télétravail non renseigné,[],Non Spécifié,Non Spécifié,https://www.welcometothejungle.com/fr/companie...
